# 线性回归的简洁实现

In [1]:
import d2l
from mxnet import autograd, np, npx, gluon
npx.set_np()

true_w = np.array([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

读取数据。

In [2]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = gluon.data.ArrayDataset(*data_arrays)
    return gluon.data.DataLoader(dataset, batch_size, shuffle=is_train)
    
batch_size = 10
data_iter = load_array((features, labels), batch_size)
for X, y in data_iter:
    print('X =\n%sy =\n%s' %(X, y))
    break

X =
[[ 0.8370042  -1.1026353 ]
 [-1.2860336  -1.6586353 ]
 [-0.591276   -1.2689118 ]
 [ 1.1089611   1.827097  ]
 [-2.1164808  -1.1797674 ]
 [ 0.4593352  -0.20153503]
 [-0.16823442 -0.38846034]
 [ 0.5477088  -1.7779099 ]
 [-1.8187165  -1.2048249 ]
 [ 1.0532789   0.24552767]]y =
[ 9.621203    7.265286    7.3253717   0.20482737  3.9688      5.806935
  5.183671   11.342728    4.669879    5.468723  ]


定义模型和初始化模型参数。

In [3]:
from mxnet.gluon import nn
from mxnet import init

net = nn.Sequential()
net.add(nn.Dense(1))
net.initialize(init.Normal(sigma=0.01))

定义损失函数和优化函数。

In [4]:
from mxnet import gluon

loss = gluon.loss.L2Loss() 
trainer = gluon.Trainer(net.collect_params(),
                        'sgd', {'learning_rate': 0.03})

训练。

In [5]:
for epoch in range(1, 4):
    for X, y in data_iter:
        with autograd.record():
            l = loss(net(X), y)
        l.backward()
        trainer.step(batch_size)
    l = loss(net(features), labels)
    print('epoch %d, loss: %f' % (epoch, l.mean()))
    
w = net[0].weight.data()
print('Error in estimating w', true_w.reshape(w.shape) - w)
b = net[0].bias.data()
print('Error in estimating b', true_b - b)    

epoch 1, loss: 0.040415
epoch 2, loss: 0.000156
epoch 3, loss: 0.000051
Error in estimating w [[ 0.000283   -0.00072527]]
Error in estimating b [0.00046492]
